# 0. Início: importar bibliotecas, inicializar funçoes auxiliares

In [1]:
import pandas as pd
from pathlib import Path
import datetime
import plotly.express as px

pd.set_option('display.max_columns', None)

### Funções auxiliares gerais

In [2]:
def countNones(df: pd.DataFrame):
    return df.isna().sum()

### Funções Auxiliares de dados de município

In [3]:
def getRegiao(data_municipios, cod_municipio_sem_digito, type="meso"):
    if type == "meso":
        key = "Nome_Mesorregião"
    elif type == "micro":
        key = "Nome_Microrregião"
    else:
        raise ValueError("type must be 'meso' or 'micro'")

    return data_municipios[data_municipios["Código Município Sem Dígito"] == cod_municipio_sem_digito][key].values[0]

def getCidade(data_municipios, cod_municipio_sem_digito):
    return data_municipios[data_municipios["Código Município Sem Dígito"] == cod_municipio_sem_digito]["Nome_Município"].values[0]

### Funções auxiliares de dados de saúde

In [4]:
def filter_last_month_of_year(data):
    assert "ANO" in data.columns and "MES" in data.columns, "data must have columns ANO and MES"
    return data.groupby(["ANO"]).apply(lambda x_ano: x_ano[x_ano["MES"] == x_ano["MES"].max()]).reset_index(drop=True)

# 1. Carregar, visualizar e preparar dados

### 1.1 Dados de munícipois (IBGE)

Esses dados serão utilizados para consultar a mesoregião de cada município por meio do seu código, e assim fazer análise dos leitos por mesoregião. 

São 5 as mesoregiões consideradas pelo IBGE para o estado de Goiás: Sul, Leste, Norte, Centro, Noroeste. Goiânia está localizada no Centro goiano.

In [5]:
DATA_MUNICIPIOS_PATH = Path("./data/RELATORIO_DTB_BRASIL_MUNICIPIO.csv")
data_municipios = pd.read_csv(DATA_MUNICIPIOS_PATH, sep=",", encoding="utf-8")
data_municipios = data_municipios[data_municipios["Nome_UF"] == "Goiás"]
data_municipios["Código Município Sem Dígito"] = data_municipios["Código Município Completo"].apply(lambda x: int(str(x)[:-1]))
data_municipios.head()

,UF,Nome_UF,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município,Código Município Sem Dígito
5323,52,Goiás,5201,Goiânia,520001,Goiânia,3,Centro Goiano,10,Goiânia,50,5200050,Abadia de Goiás,520005
5324,52,Goiás,5201,Goiânia,520002,Anápolis,4,Leste Goiano,12,Entorno de Brasília,100,5200100,Abadiânia,520010
5325,52,Goiás,5204,São Luís de Montes Belos - Iporá,520015,Palmeiras de Goiás,5,Sul Goiano,14,Vale do Rio dos Bois,134,5200134,Acreúna,520013
5326,52,Goiás,5201,Goiânia,520003,Inhumas - Itaberaí - Anicuns,3,Centro Goiano,9,Anicuns,159,5200159,Adelândia,520015
5327,52,Goiás,5206,Luziânia - Águas Lindas de Goiás,520022,Flores de Goiás,4,Leste Goiano,12,Entorno de Brasília,175,5200175,Água Fria de Goiás,520017


### 1.2 Dados de leitos em Goiás (CNES)

### Visualização da tabela

In [6]:
DATA_PATH = Path('./data/data_original_2017_to_2022.csv')
data = pd.read_csv(DATA_PATH, index_col="Unnamed: 0")
data

/tmp/ipykernel_65373/1661828260.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(DATA_PATH, index_col="Unnamed: 0")


,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,CNPJ_MAN,ESFERA_A,ATIVIDAD,RETENCAO,NATUREZA,CLIENTEL,TP_UNID,TURNO_AT,NIV_HIER,TERCEIRO,TP_LEITO,CODLEITO,QT_EXIST,QT_CONTR,QT_SUS,QT_NSUS,COMPETEN,NAT_JUR
1,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,2,41,2,0,2,0,201701,2062
2,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,6,34,4,0,3,1,201701,2062
3,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,2,33,6,0,4,2,201701,2062
4,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,4,43,3,0,3,0,201701,2062
5,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,4,10,1,0,1,0,201701,2062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206188,2383691,522205,001,NaN,NaN,NaN,M,3,0,3,44834000107,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,1,3,3,0,3,0,202211,1244
206189,2383691,522205,001,NaN,NaN,NaN,M,3,0,3,44834000107,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,4,10,3,0,3,0,202211,1244
206190,2439883,522220,02,NaN,NaN,NaN,M,3,0,3,37388378000114,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,2,33,7,0,7,0,202211,1244
206191,2439883,522220,02,NaN,NaN,NaN,M,3,0,3,37388378000114,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,4,43,2,0,2,0,202211,1244


### Visualização das colunas da tabela

Ver documento info/IT_CNES_1706.pdf para informações sobrecada coluna.

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206192 entries, 1 to 206192
Data columns (total 28 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   CNES      206192 non-null  int64  
 1   CODUFMUN  206192 non-null  int64  
 2   REGSAUDE  126342 non-null  object 
 3   MICR_REG  11221 non-null   object 
 4   DISTRSAN  122 non-null     float64
 5   DISTRADM  2175 non-null    object 
 6   TPGESTAO  206192 non-null  object 
 7   PF_PJ     206192 non-null  int64  
 8   CPF_CNPJ  206192 non-null  int64  
 9   NIV_DEP   206192 non-null  int64  
 10  CNPJ_MAN  206192 non-null  int64  
 11  ESFERA_A  0 non-null       float64
 12  ATIVIDAD  206192 non-null  int64  
 13  RETENCAO  0 non-null       float64
 14  NATUREZA  0 non-null       float64
 15  CLIENTEL  205097 non-null  float64
 16  TP_UNID   206192 non-null  int64  
 17  TURNO_AT  206192 non-null  int64  
 18  NIV_HIER  0 non-null       float64
 19  TERCEIRO  0 non-null       float64
 20  TP_L

### Meses para os quiais temos dados

Todos entre janeiro de 2017 a novembro de 2021

In [8]:
print(data["COMPETEN"].nunique())
periods = data["COMPETEN"].unique()
periods

71


array([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
       201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
       201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812,
       201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
       202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
       202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108,
       202109, 202110, 202111, 202112, 202201, 202202, 202203, 202204,
       202205, 202206, 202207, 202208, 202209, 202210, 202211])

### Adição de colunas auxiliares nos dados:
- Ano
- Mês
- Mesoregião
- Cidade

In [9]:
data["COMPETEN"] = data["COMPETEN"].astype(str)
data["ANO"] = data["COMPETEN"].apply(lambda x: x[:4])
data["MES"] = data["COMPETEN"].apply(lambda x: x[4:])
# May take a while (+ 25 secs)
data["NOME_MUNICIPIO"] = data["CODUFMUN"].apply(lambda x: getCidade(data_municipios, x))
# May take a while (+ 25 secs)
data["MESOREGIAO"] = data["CODUFMUN"].apply(lambda x: getRegiao(data_municipios, x, "meso"))
data

,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,CNPJ_MAN,ESFERA_A,ATIVIDAD,RETENCAO,NATUREZA,CLIENTEL,TP_UNID,TURNO_AT,NIV_HIER,TERCEIRO,TP_LEITO,CODLEITO,QT_EXIST,QT_CONTR,QT_SUS,QT_NSUS,COMPETEN,NAT_JUR,ANO,MES,NOME_MUNICIPIO,MESOREGIAO
1,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,2,41,2,0,2,0,201701,2062,2017,01,Acreúna,Sul Goiano
2,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,6,34,4,0,3,1,201701,2062,2017,01,Acreúna,Sul Goiano
3,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,2,33,6,0,4,2,201701,2062,2017,01,Acreúna,Sul Goiano
4,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,4,43,3,0,3,0,201701,2062,2017,01,Acreúna,Sul Goiano
5,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,4,10,1,0,1,0,201701,2062,2017,01,Acreúna,Sul Goiano
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206188,2383691,522205,001,NaN,NaN,NaN,M,3,0,3,44834000107,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,1,3,3,0,3,0,202211,1244,2022,11,Vicentinópolis,Sul Goiano
206189,2383691,522205,001,NaN,NaN,NaN,M,3,0,3,44834000107,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,4,10,3,0,3,0,202211,1244,2022,11,Vicentinópolis,Sul Goiano
206190,2439883,522220,02,NaN,NaN,NaN,M,3,0,3,37388378000114,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,2,33,7,0,7,0,202211,1244,2022,11,Vila Boa,Leste Goiano
206191,2439883,522220,02,NaN,NaN,NaN,M,3,0,3,37388378000114,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,4,43,2,0,2,0,202211,1244,2022,11,Vila Boa,Leste Goiano


In [10]:
# Visualizando mesoregiões para as quais temos dados
data["MESOREGIAO"].unique()

array(['Sul Goiano', 'Leste Goiano', 'Norte Goiano', 'Centro Goiano',
       'Noroeste Goiano'], dtype=object)

In [11]:
CODIGOS_UTI = [74, 75, 76, 77, 78, 80, 81, 82, 83, 85, 86]
MAP_TIPOS_LEITO = {
    "CIRURGICO": 1,
    "CLINICO": 2,
    "COMPLEMENTARES": 3,    # UTIS e Unidades Intermediárias estão aqui
    "OBSTETRICO": 4,
    "PEDIATRICO": 5,
    "OUTRAS ESPECIALIDADES": 6,
    "HOSPITAL DIA": 7,
}

data_UTI = data[data["CODLEITO"].isin(CODIGOS_UTI)]
data_UTI.head()

,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,CNPJ_MAN,ESFERA_A,ATIVIDAD,RETENCAO,NATUREZA,CLIENTEL,TP_UNID,TURNO_AT,NIV_HIER,TERCEIRO,TP_LEITO,CODLEITO,QT_EXIST,QT_CONTR,QT_SUS,QT_NSUS,COMPETEN,NAT_JUR,ANO,MES,NOME_MUNICIPIO,MESOREGIAO
74,2361787,520110,3,NaN,NaN,NaN,M,3,1038751000160,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,3,75,20,0,16,4,201701,3999,2017,01,Anápolis,Centro Goiano
75,2361787,520110,3,NaN,NaN,NaN,M,3,1038751000160,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,3,78,10,0,10,0,201701,3999,2017,01,Anápolis,Centro Goiano
76,2361787,520110,3,NaN,NaN,NaN,M,3,1038751000160,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,3,81,10,0,10,0,201701,3999,2017,01,Anápolis,Centro Goiano
107,2442108,520110,3,NaN,NaN,NaN,M,3,1020197000193,3,36975290000136,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,3,78,8,0,6,2,201701,2054,2017,01,Anápolis,Centro Goiano
108,2442108,520110,3,NaN,NaN,NaN,M,3,1020197000193,3,36975290000136,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,3,75,34,0,15,19,201701,2054,2017,01,Anápolis,Centro Goiano


# 2. Análise da evolução da quantidade de Leitos em Goiás (Todos os tipos de Leito)

## Evolução da quantidade de leitos por mês

In [12]:
leitos_por_mes = data.groupby(["COMPETEN"]).sum()[["QT_EXIST", "QT_SUS"]].reset_index()
leitos_por_mes

,COMPETEN,QT_EXIST,QT_SUS
0,201701,19490,12122
1,201702,19540,12147
2,201703,19593,11939
3,201704,19636,11865
4,201705,19570,11820
...,...,...,...
66,202207,21303,12819
67,202208,21343,12988
68,202209,21315,12942
69,202210,21389,13096


In [13]:
px.line(leitos_por_mes, x="COMPETEN", y=["QT_EXIST", "QT_SUS"], title="Quantidade de leitos por mês", range_y=[0, 24000])

## Evolução da quantidade de Leitos ao final de cada ano

In [14]:
data_ultimo_mes = filter_last_month_of_year(data)
print(data_ultimo_mes["COMPETEN"].unique())
data_ultimo_mes

['201712' '201812' '201912' '202012' '202112' '202211']


,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,CNPJ_MAN,ESFERA_A,ATIVIDAD,RETENCAO,NATUREZA,CLIENTEL,TP_UNID,TURNO_AT,NIV_HIER,TERCEIRO,TP_LEITO,CODLEITO,QT_EXIST,QT_CONTR,QT_SUS,QT_NSUS,COMPETEN,NAT_JUR,ANO,MES,NOME_MUNICIPIO,MESOREGIAO
0,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,2,33,6,0,4,2,201712,2062,2017,12,Acreúna,Sul Goiano
1,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,4,43,3,0,3,0,201712,2062,2017,12,Acreúna,Sul Goiano
2,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,2,41,2,0,2,0,201712,2062,2017,12,Acreúna,Sul Goiano
3,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,5,45,3,0,3,0,201712,2062,2017,12,Acreúna,Sul Goiano
4,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,6,34,4,0,3,1,201712,2062,2017,12,Acreúna,Sul Goiano
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17434,2383691,522205,001,NaN,NaN,NaN,M,3,0,3,44834000107,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,1,3,3,0,3,0,202211,1244,2022,11,Vicentinópolis,Sul Goiano
17435,2383691,522205,001,NaN,NaN,NaN,M,3,0,3,44834000107,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,4,10,3,0,3,0,202211,1244,2022,11,Vicentinópolis,Sul Goiano
17436,2439883,522220,02,NaN,NaN,NaN,M,3,0,3,37388378000114,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,2,33,7,0,7,0,202211,1244,2022,11,Vila Boa,Leste Goiano
17437,2439883,522220,02,NaN,NaN,NaN,M,3,0,3,37388378000114,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,4,43,2,0,2,0,202211,1244,2022,11,Vila Boa,Leste Goiano


In [15]:
leitos_ultimo_mes = data_ultimo_mes.groupby(["COMPETEN", "ANO"]).sum()[["QT_EXIST", "QT_SUS"]].reset_index()
px.line(leitos_ultimo_mes, x="ANO", y=["QT_EXIST", "QT_SUS"], title="Quantidade de leitos no último mês do ano")

## Evolução da quantidade de Leitos por Mesoregião

### Total

In [16]:
leitos_por_regiao = data.groupby(["COMPETEN", "MESOREGIAO"]).sum()[["QT_EXIST", "QT_SUS"]].reset_index()
px.line(leitos_por_regiao, x="COMPETEN", y="QT_EXIST", color="MESOREGIAO", title="Quantidade de leitos totais por mesoregião", range_y=[0, 16000])

In [17]:
data_ultimo_mes_por_meso = data.groupby(["COMPETEN", "MESOREGIAO", "ANO", "MES"]).sum()[["QT_EXIST", "QT_SUS"]].reset_index()
data_ultimo_mes_por_meso = filter_last_month_of_year(data_ultimo_mes_por_meso)

In [18]:
px.bar(data_ultimo_mes_por_meso, x="ANO", y="QT_EXIST", color="MESOREGIAO", title="Quantidade de leitos totais por mesoregião no final de cada ano", barmode="group")

### Sus

In [19]:
px.line(leitos_por_regiao, x="COMPETEN", y="QT_SUS", color="MESOREGIAO", title="Quantidade de leitos SUS por mesoregião", range_y=[0, 10000])

In [20]:
px.bar(data_ultimo_mes_por_meso, x="ANO", y="QT_SUS", color="MESOREGIAO", title="Quantidade de leitos SUS por mesoregião", barmode="group")

## Evolução da quantidade de Leitos por Cidade

- A fim de simplificar a visualização, vamos considerar apenas as 10 cidades com mais leitos em dezembro de 2021

In [21]:
leitos_por_cidade = data.groupby(["COMPETEN", "NOME_MUNICIPIO", "ANO", "MES"]).sum()[["QT_EXIST", "QT_SUS"]].reset_index()
leitos_por_cidade_fim_ano = filter_last_month_of_year(leitos_por_cidade)


In [22]:
leitos_fim_2021_por_cidade = leitos_por_cidade_fim_ano[leitos_por_cidade_fim_ano["ANO"] == "2021"]
leitos_fim_2021_por_cidade.sort_values(by="QT_EXIST", ascending=False).head(10)

,COMPETEN,NOME_MUNICIPIO,ANO,MES,QT_EXIST,QT_SUS
830,202112,Goiânia,2021,12,9306,4161
763,202112,Aparecida de Goiânia,2021,12,1700,1337
762,202112,Anápolis,2021,12,1364,748
907,202112,Rio Verde,2021,12,648,400
932,202112,Trindade,2021,12,555,444
938,202112,Uruaçu,2021,12,402,252
792,202112,Catalão,2021,12,397,218
796,202112,Ceres,2021,12,363,146
850,202112,Itumbiara,2021,12,284,167
827,202112,Goianésia,2021,12,283,195


In [23]:
cidades_com_mais_leitos_dez_2021 = leitos_fim_2021_por_cidade.sort_values(by="QT_EXIST", ascending=False).head(10)["NOME_MUNICIPIO"].unique()
print(cidades_com_mais_leitos_dez_2021)
leitos_por_cidade_fim_ano = leitos_por_cidade_fim_ano[leitos_por_cidade_fim_ano["NOME_MUNICIPIO"].isin(cidades_com_mais_leitos_dez_2021)]
leitos_por_cidade_fim_ano.head()

['Goiânia' 'Aparecida de Goiânia' 'Anápolis' 'Rio Verde' 'Trindade'
 'Uruaçu' 'Catalão' 'Ceres' 'Itumbiara' 'Goianésia']


,COMPETEN,NOME_MUNICIPIO,ANO,MES,QT_EXIST,QT_SUS
8,201712,Anápolis,2017,12,1155,618
9,201712,Aparecida de Goiânia,2017,12,871,666
38,201712,Catalão,2017,12,347,226
42,201712,Ceres,2017,12,312,136
73,201712,Goianésia,2017,12,271,172


In [24]:
px.bar(leitos_por_cidade_fim_ano, x="ANO", y="QT_EXIST", color="NOME_MUNICIPIO", title="Quantidade de leitos totais por cidade no final de cada ano", barmode="group")

# 3. Análise da evolução da quantidade de Leitos em Goiás (UTI)

## Evolução da quantidade de leitos por mês

In [25]:
leitos_por_mes = data_UTI.groupby(["COMPETEN"]).sum()[["QT_EXIST", "QT_SUS"]].reset_index()
leitos_por_mes

,COMPETEN,QT_EXIST,QT_SUS
0,201701,1355,722
1,201702,1375,722
2,201703,1414,728
3,201704,1456,728
4,201705,1456,728
...,...,...,...
66,202207,2081,1019
67,202208,2114,1019
68,202209,2115,1019
69,202210,2107,1020


In [26]:
px.line(leitos_por_mes, x="COMPETEN", y=["QT_EXIST", "QT_SUS"], title="Quantidade de leitos por mês", range_y=[0, 2400])

## Evolução da quantidade de Leitos ao final de cada ano

In [27]:
data_ultimo_mes = filter_last_month_of_year(data_UTI)
print(data_ultimo_mes["COMPETEN"].unique())
data_ultimo_mes

['201712' '201812' '201912' '202012' '202112' '202211']


,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,CNPJ_MAN,ESFERA_A,ATIVIDAD,RETENCAO,NATUREZA,CLIENTEL,TP_UNID,TURNO_AT,NIV_HIER,TERCEIRO,TP_LEITO,CODLEITO,QT_EXIST,QT_CONTR,QT_SUS,QT_NSUS,COMPETEN,NAT_JUR,ANO,MES,NOME_MUNICIPIO,MESOREGIAO
0,2361787,520110,3,NaN,NaN,NaN,M,3,1038751000160,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,3,75,20,0,16,4,201712,3999,2017,12,Anápolis,Centro Goiano
1,2361787,520110,3,NaN,NaN,NaN,M,3,1038751000160,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,3,81,10,0,10,0,201712,3999,2017,12,Anápolis,Centro Goiano
2,2361787,520110,3,NaN,NaN,NaN,M,3,1038751000160,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,3,78,10,0,10,0,201712,3999,2017,12,Anápolis,Centro Goiano
3,2442108,520110,3,NaN,NaN,NaN,M,3,36975290000136,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,3,78,8,0,6,2,201712,3069,2017,12,Anápolis,Centro Goiano
4,2442108,520110,3,NaN,NaN,NaN,M,3,36975290000136,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,3,75,34,0,15,19,201712,3069,2017,12,Anápolis,Centro Goiano
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,2570777,522045,1,NaN,NaN,NaN,M,3,2485436000143,1,0,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,3,75,14,0,8,6,202211,2062,2022,11,Senador Canedo,Centro Goiano
798,5095808,522140,1,NaN,NaN,NaN,E,3,0,3,2529964000157,NaN,4,NaN,NaN,3.0,5,6,NaN,NaN,3,75,10,0,0,10,202211,1023,2022,11,Trindade,Centro Goiano
799,547484,522160,NaN,NaN,NaN,NaN,E,3,0,3,2529964000157,NaN,4,NaN,NaN,2.0,5,6,NaN,NaN,3,78,10,0,10,0,202211,1023,2022,11,Uruaçu,Norte Goiano
800,547484,522160,NaN,NaN,NaN,NaN,E,3,0,3,2529964000157,NaN,4,NaN,NaN,2.0,5,6,NaN,NaN,3,81,10,0,0,10,202211,1023,2022,11,Uruaçu,Norte Goiano


In [28]:
leitos_ultimo_mes = data_ultimo_mes.groupby(["COMPETEN", "ANO"]).sum()[["QT_EXIST", "QT_SUS"]].reset_index()
px.line(leitos_ultimo_mes, x="ANO", y=["QT_EXIST", "QT_SUS"], title="Quantidade de leitos no último mês do ano")

## Evolução da quantidade de Leitos por Mesoregião

### Total

In [29]:
leitos_por_regiao = data_UTI.groupby(["COMPETEN", "MESOREGIAO"]).sum()[["QT_EXIST", "QT_SUS"]].reset_index()
px.line(leitos_por_regiao, x="COMPETEN", y="QT_EXIST", color="MESOREGIAO", title="Quantidade de leitos totais por mesoregião", range_y=[0, 2000])

In [30]:
data_ultimo_mes_por_meso = data_UTI.groupby(["COMPETEN", "MESOREGIAO", "ANO", "MES"]).sum()[["QT_EXIST", "QT_SUS"]].reset_index()
data_ultimo_mes_por_meso = filter_last_month_of_year(data_ultimo_mes_por_meso)

In [31]:
px.bar(data_ultimo_mes_por_meso, x="ANO", y="QT_EXIST", color="MESOREGIAO", title="Quantidade de leitos totais por mesoregião no final de cada ano", barmode="group")

### Sus

In [32]:
px.line(leitos_por_regiao, x="COMPETEN", y="QT_SUS", color="MESOREGIAO", title="Quantidade de leitos SUS por mesoregião", range_y=[0, 1000])

In [33]:
px.bar(data_ultimo_mes_por_meso, x="ANO", y="QT_SUS", color="MESOREGIAO", title="Quantidade de leitos SUS por mesoregião", barmode="group")

## Evolução da quantidade de Leitos por Cidade

- A fim de simplificar a visualização, vamos considerar apenas as 10 cidades com mais leitos em dezembro de 2021

In [34]:
leitos_por_cidade = data_UTI.groupby(["COMPETEN", "NOME_MUNICIPIO", "ANO", "MES"]).sum()[["QT_EXIST", "QT_SUS"]].reset_index()
leitos_por_cidade_fim_ano = filter_last_month_of_year(leitos_por_cidade)


In [35]:
leitos_fim_2021_por_cidade = leitos_por_cidade_fim_ano[leitos_por_cidade_fim_ano["ANO"] == "2021"]
leitos_fim_2021_por_cidade.sort_values(by="QT_EXIST", ascending=False).head(10)

,COMPETEN,NOME_MUNICIPIO,ANO,MES,QT_EXIST,QT_SUS
83,202112,Goiânia,2021,12,1100,461
75,202112,Anápolis,2021,12,162,81
76,202112,Aparecida de Goiânia,2021,12,119,86
92,202112,Rio Verde,2021,12,91,19
89,202112,Nerópolis,2021,12,52,48
78,202112,Catalão,2021,12,34,20
79,202112,Ceres,2021,12,29,4
85,202112,Jataí,2021,12,20,17
93,202112,Santa Helena de Goiás,2021,12,20,20
97,202112,Uruaçu,2021,12,20,0


In [36]:
cidades_com_mais_leitos_dez_2021 = leitos_fim_2021_por_cidade.sort_values(by="QT_EXIST", ascending=False).head(10)["NOME_MUNICIPIO"].unique()
print(cidades_com_mais_leitos_dez_2021)
leitos_por_cidade_fim_ano = leitos_por_cidade_fim_ano[leitos_por_cidade_fim_ano["NOME_MUNICIPIO"].isin(cidades_com_mais_leitos_dez_2021)]
leitos_por_cidade_fim_ano.head()

['Goiânia' 'Anápolis' 'Aparecida de Goiânia' 'Rio Verde' 'Nerópolis'
 'Catalão' 'Ceres' 'Jataí' 'Santa Helena de Goiás' 'Uruaçu']


,COMPETEN,NOME_MUNICIPIO,ANO,MES,QT_EXIST,QT_SUS
0,201712,Anápolis,2017,12,110,81
1,201712,Aparecida de Goiânia,2017,12,89,61
3,201712,Catalão,2017,12,28,15
4,201712,Ceres,2017,12,20,4
7,201712,Goiânia,2017,12,916,466


In [37]:
px.bar(leitos_por_cidade_fim_ano, x="ANO", y="QT_EXIST", color="NOME_MUNICIPIO", title="Quantidade de leitos totais por cidade no final de cada ano", barmode="group")